In [ ]:
# This script is made to gernerate science maps from MaNGA datacubes using Marvin software

In [ ]:
# Check my current environment
!conda info --envs

In [ ]:
import numpy as np
from astropy.io import fits
from marvin.tools.maps import Maps
from marvin.tools.image import Image
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import glob
import pandas as pd
from marvin import config
config.setRelease('DR16')
#import plotly.express as px
#import plotly.graph_objects as go

## Masking function using the map data mask value to replace bad value with np.nan

In [ ]:
def map_masking(map_arr, mask_arr, fill_val=np.nan, real_val=0):
    """
    Masking function to replace bad values (>0) of a 2d np.ndarray with a fill value

    Code:
        map_arr[mask_arr != real_val] = fill_val


    Parameters:
        map_arr: ndarray
            2D ndarray with original values

        mask_arr: ndarray
            2D ndarray of the mask array and must be the same shape as map_arr

        fill_val: int, float, np,nan, np.inf (Default=np.nan)
            The value that will be substituted for the bad values in the map array

        real_val: int or float (Default=0)
            The value that repersents good value in the mask array
    """

    map_arr[mask_arr != real_val] = fill_val

    return map_arr

# Balmer Decrement

In [ ]:
def balmer_dec(obs_haflux, obs_hbflux):
    '''
    Calculates Balmer decrement (Ha_flux / Hb_flux)
    
    Paremeters:
        obs_haflux: float, numpy.ndarray, marvin.tools.quantities.map.Map 
            Observed Halpha flux value or Map
        
        obs_hbflux: float, numpy.ndarray, marvin.tools.quantities.map.Map
            Observed Hbeta flux value or Map
    '''
    bdec = obs_haflux / obs_hbflux
    return bdec


# Calzetti extinction curve 2000 Function 

In [ ]:
def c00_k(wavelength, Rv=4.05):
    '''
    Calzetti extinction curve 2000 (Good for starburst like dust)
    Paremeters:
    
        wavelength: float, 
            units of microns
        
        Rv: float, 
            normilaztion of the extinction curve
    '''
    if np.logical_and(wavelength >= 0.63, wavelength <= 2.2):
        #k_lambda = 1.17 * (-1.857 + (1.040/rest_lam)) + 1.78 # Calzetti 2001 obscuration code
        k_lambda = 2.659 * (-1.857 + (1.040/wavelength)) + Rv # Calzetti 2000
        #print('k_lambda={}'.format(k_lambda))
    elif np.logical_and(wavelength >= 0.12, wavelength < 0.63):
        #k_lambda = 1.17 * (-2.156 + (1.509/rest_lam) - (0.198/rest_lam**2) + (0.011/rest_lam**3)) + 1.78 # Calzetti 2001 obscuration code
        k_lambda = 2.659 * (-2.156 + (1.509/wavelength) - (0.198/wavelength**2) + (0.011/wavelength**3)) + Rv
        #print('k_lambda={}'.format(k_lambda))
    else:
        print('Rest wavelength is not in range or not in Angstroms {}'.format(wavelength))
        
    return k_lambda




def c00_kcorr(obs_flux, obs_wavelength, bdec, Rv=4.05):
    '''
    Extinction correction using the Calzetti extinction curve
    
    Paremeter:
        obs_flux: float, numpy.ndarray, marvin.tools.quantities.map.Map
            Obsereved flux value or map [1e-17erg/cm^2/s/spaxel]
        
        obs_wavelength: float, 
            wavelength [micron]

        bdec: float, numpy.ndarray, marvin.tools.quantities.map.Map
            Balmer Decrement (Ha/Hb)
            *Must be same type as obs_flux*
        
        Rv: float, curve normilization
    '''
    
    # Observe flux
    #print('Observe Flux = {} 1e-17erg/cm^2/spaxel/s'.format(obs_flux))
    
    # Observe extinction curve 
    obs_k = c00_k(obs_wavelength, Rv=Rv)
    #print('k({} mircon) = {}'.format(obs_wavelength, obs_k))
    
    # Ha, Hb extinction curve
    k_Ha = c00_k(.6564, Rv)
    #print('k({} mircon) = {}'.format(.6564, k_Ha))
    k_Hb = c00_k(.4864, Rv)
    #print('k({} mircon) = {}'.format(.4864, k_Hb))
    
    # Optical depth tau
    tau = np.log10(bdec / 2.86) # possibly np.log10
    #print('Balmer optical depth={}'.format(tau))
    
    # color excess(gas) - Battisti et al 2017 eqn 2
    EBV_gas = (1.086 / (k_Hb - k_Ha)) * tau 
    EBV_star = EBV_gas * 0.44
    #print('E(B-V)gas={}'.format(EBV_gas))
    #print('E(B-V)star={}'.format(EBV_star))
    
    # Intrinsic flux [erg/cm^2/s]
    int_flux = obs_flux * 10**(0.4 * obs_k * EBV_star)
    #print('intrinsic_flux = {} 1e-17erg/cm^2/s'.format(int_flux))
    #print('')
    
    return int_flux

# Star formation rate density

In [ ]:
def sfr_map2(halphadc_map, z):
    """
    Paremters:
        halphadc_map
    
    """
    #sfr_map = (7.9e-42 * halphadc_map) / 1.53
    
    # Calculate Luminosity distance
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
    lum_d_mpc = cosmo.luminosity_distance(z)
    lum_d_cm = lum_d_mpc.to(u.cm).value # convert from Mpc to cm
    #print('Lum Distance {} converted to {} cm'.format(lum_d_mpc, lum_d_cm))
    
    #Convert flux to Luminosity
    l_ha = halphadc_map * (4 * np.pi * (lum_d_cm**2)) #1e-17 erg / (s spaxel)
    #print('Halpha Flux: ',halphadc_map[27,27])
    #print('Halpha Luminosity: ',l_ha[27,27])
    
    #sfr - Salpeter initial mass function Hao et al. (2011)
    sfr_map = (8.79e-42 * l_ha) # [Msolar/yr]
    #print('SFR: ',sfr_map[27,27])
    
    spaxel_size = 0.5  # [arcsec]
    c = 299792  # speed of light [km/s]
    H0 = 70  # [km s^-1 Mpc^-1]
    D = c * z / H0  # approx. distance to galaxy [Mpc]

    scale = 1 / 206265 * D * 1e6  # 1 radian = 206265 arcsec [pc / arcsec]
    spaxel_area_pc = (scale * spaxel_size)**2 * u.pc**2 # [pc^2]
    spaxel_area_kpc = spaxel_area_pc.to(u.kpc**2)

    #Calculate the SFRD of each spaxel
    sfrd_map = sfr_map / spaxel_area_kpc.value

    ##Measeure the diameter of each spaxel with respect to the galaxy distance
    #spaxel_diamter_in_kpc = cosmo.kpc_proper_per_arcmin(z).to(u.kpc/u.arcsec) * (0.5 * u.arcsec)
    ##print('Diamter of each spaxel (0.5 arcsec) on the galaxy: {}'.format(spaxel_diamter_in_kpc))
    #
    ##Area of each spaxel
    #Area_of_each_spaxels =  4 * np.pi * (spaxel_diamter_in_kpc/2)**2
    ##print('Area of each spaxel: {}'.format(Area_of_each_spaxels))
    #
    ##Calculate the SFRD of each spaxel
    #sfrd_map = sfr_map / Area_of_each_spaxels
    ##print('SFRD: ',sfrd_map[27,27])
    
    return sfr_map, l_ha, sfrd_map, spaxel_area_kpc

# Radius Map and effective radius from Marvin

In [ ]:
def radius_ratio_map(plateifu):
    '''
    platifu: str 
    
    '''
    #Galaxy effective radius
    r_eff = float(Maps(plateifu=plateifu).header['reff']) #arcsec
    #print('Effective Radius', Reff)
    
    # Galaxy elliptical radius
    r_map = Maps(plateifu=plateifu).spx_ellcoo_elliptical_radius.value #arcsec
    #print('Elliptical Radius', R)
    
    #R/Reff
    radius_ratio = r_map / r_eff
    
    return r_map, radius_ratio, r_eff

# Metallicity Maps functions

In [ ]:
def n2o2_Z_map(nii6585_fluxmap, oii3727_fluxmap, oii3729_fluxmap):
    # Kewley and Dopita - 2002 - Using Strong Lines to Estimate Abundances in Extra (KD02)
    # NII/OII diagnostic [N II] λ6584/[O II] λ3727,3729
    # Note: Independent of ionization parameter, Z>=8.6 for a reliable abundance
    fratio_map = np.log10(nii6585_fluxmap / (oii3727_fluxmap+oii3729_fluxmap))
    Z_map = np.log10(1.54020 + (1.26602 * fratio_map) + (0.167977 * fratio_map**2)) + 8.93 # KD02 (eq 5&7) [Z = log(O/H) +12]
    return fratio_map, Z_map

def o3n2_metal_map(o3_5008_fluxmap, nii6585_fluxmap, ha_fluxmap, hb_fluxmap):
    # Marino et al. 2013 O3N2 diagnostic
    # O3N2 diagnostic [OIII]λ5007/Hbeta * Halpha/[NII]λ6583
    fratio_map = np.log10((o3_5008_fluxmap/hb_fluxmap) * (ha_fluxmap / nii6585_fluxmap))
    Z_map = 8.505 - 0.221 * fratio_map
    return fratio_map, Z_map

def n2_metal_map(nii6585_fluxmap, ha_fluxmap):
    # Marino et al. 2013 N2 diagnostic
    # N2 diagnostic log([NII]λ6583/Halpha)
    fratio_map = np.log10(nii6585_fluxmap / ha_fluxmap)
    Z_map = 8.667 - 0.455 * fratio_map # Log[O/H] + 12
    return fratio_map, Z_map

#def n2s2_metal_map(nii6585_fluxmap, sii6718_fluxmap, sii6732_fluxmap, ha_fluxmap):
#    # Marino et al. 2013 N2 diagnostic
#    # N2S2 diagnostic log([NII]λ6583/Halpha)
#    n2_ratio_map, n2_Z_map = n2_metal_map(nii6585_fluxmap, ha_fluxmap) # 
#    fratio_map = np.log10(nii6585_fluxmap / (sii6718_fluxmap + sii6732_fluxmap))
#    metal_map = 8.77 + fratio_map + 0.264 * n2_Z_map
#    return ratio_map, metal_map

# Pipe3D Maps

In [ ]:
# Pipe3D Maps
def pipe3d_maps(plateifu, sample='bbrd'):
    '''
    Parameters:
        plateifu: str
            MaNGA plateifu
    
    Return:
    age_l, age_Z, age_err, stel_vel, vel_err, v_disp, v_err, ml_ratio, mass_density
    '''
    #SSP
    hdu = fits.open('/Users/mmckay/Desktop/research/FMR_MZR/{}_pipe3d_fits/manga-{}.Pipe3D.cube.fits'.format(sample, plateifu))
    age_l = hdu[1].data[5,:,:] # Gyr - Luminosity Weighted age
    age_m = hdu[1].data[6,:,:] # Gyr - Mass Weighted age 
    age_err =  hdu[1].data[7,:,:] # Gyr - Error of the age
    metal_l = hdu[1].data[8,:,:] # Luminosity Weighted metallicity of the stellar population (where Z=0.02 is solar metallicity)
    metal_m = hdu[1].data[9,:,:] # Mass Weighted metallicity of the stellar population
    metal_err =   hdu[1].data[10,:,:] # Error of the metallicity of the stellar population
    stel_vel = hdu[1].data[13,:,:] # [km/s] Velocity map of the stellar population
    vel_err = hdu[1].data[14,:,:] # Error in the velocity of the stellar population
    v_disp = hdu[1].data[15,:,:] # Velocity dispersion of the stellar population (sigma)
    v_err = hdu[1].data[16,:,:] # Error in velocity dispersion of the stellar population
    ml_ratio = hdu[1].data[17,:,:] # [Log(Msun/Lsun)] Average mass-to-light ratio of the stellar population
    mass_rho = hdu[1].data[19,:,:] # [Log(Msun/spaxels^2)] Stellar Mass density per pixel with dust correction
    mass_rho = hdu[1].data[19,:,:] # [Log(Msun/spaxels^2)] Stellar Mass density per pixel with dust correction

    #INDCICES - not corrected for velocity dispersion
    d4000_index = hdu[4].data[5,:,:] # D4000 index map
    d4000_err = hdu[4].data[13,:,:] #D4000 index err map
    hdelta_index = hdu[4].data[6,:,:] #Hdelta index map
    hdelta_err = hdu[4].data[14,:,:] #Hdelta index map

    hdu.close()

    return age_l, age_m, age_err, metal_l, metal_m, metal_err, stel_vel, vel_err, v_disp, v_err, ml_ratio, mass_rho, d4000_index, d4000_err, hdelta_index, hdelta_err

# Generating Maps

In [ ]:
def write_maps2fits(plateifu, mode='local', sample='bbrd'):
    
    # Load Marvin Maps
    maps = Maps(plateifu=plateifu, mode=mode)

    # flux maps
    ha6564flux_map = maps.emline_gflux_ha_6564 # [1e-17 erg / (cm s spaxel)]
    hb4862flux_map = maps.emline_gflux_hb_4862
    nii6585_map = maps.emline_gflux_nii_6585
    oii3727_map = maps.emline_gflux_oii_3727
    oii3729_map = maps.emline_gflux_oii_3729
    o3_5008_map = maps.emline_gflux_oiii_5008
    sii6718_map = maps.emline_gflux_sii_6718
    sii6732_map = maps.emline_gflux_sii_6732
    snr_map = maps.spx_snr # SNR map

    # Remove bad pixels values from Maps and replaces them with a fill_val
    ha_map_clean = map_masking(map_arr=ha6564flux_map.value, mask_arr=ha6564flux_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    hb_map_clean = map_masking(map_arr=hb4862flux_map.value, mask_arr=hb4862flux_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    nii6585_map_clean = map_masking(map_arr=nii6585_map.value, mask_arr=nii6585_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    oii3727_map_clean = map_masking(map_arr=oii3727_map.value, mask_arr=oii3727_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    oii3729_map_clean = map_masking(map_arr=oii3729_map.value, mask_arr=oii3729_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    o3_5008_map_clean = map_masking(map_arr=o3_5008_map.value, mask_arr=o3_5008_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    sii6718_map_clean = map_masking(map_arr=sii6718_map.value, mask_arr=sii6718_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels
    sii6732_map_clean = map_masking(map_arr=sii6732_map.value, mask_arr=sii6732_map.mask, fill_val=np.nan, real_val=0) # remove bad spaxels


    # Balmer Decrrement
    bdec_map = balmer_dec(obs_haflux=ha_map_clean, obs_hbflux=hb_map_clean)
    bdec_map[bdec_map == np.inf] = 0.0 # Replace inf values with zero incase hbeta map had zero values

    # Run kcorrection function on emission lines
    ha_mapdc = c00_kcorr(ha_map_clean, .6564, bdec_map, Rv=4.05)
    hb_mapdc = c00_kcorr(hb_map_clean, .4864, bdec_map, Rv=4.05)
    nii6585_mapdc = c00_kcorr(nii6585_map_clean, .6585, bdec_map, Rv=4.05)
    oii3727_mapdc = c00_kcorr(oii3727_map_clean, .3727, bdec_map, Rv=4.05)
    oii3729_mapdc = c00_kcorr(oii3729_map_clean, .3729, bdec_map, Rv=4.05)
    o3_5008_mapdc = c00_kcorr(o3_5008_map_clean, .5008, bdec_map, Rv=4.05)
    sii6718_mapdc = c00_kcorr(sii6718_map_clean, .6718, bdec_map, Rv=4.05)
    sii6732_mapdc = c00_kcorr(sii6732_map_clean, .6732, bdec_map, Rv=4.05)

    # Get redshift
    z = maps.nsa['z']

    # Run Star formation code
    #sfr_map, l_ha, sfrd_map, spaxel_diamter_in_kpc, Area_of_each_spaxels = sfr_map(halphadc_map=ha_mapdc, z=z)
    sfr_map, l_ha, sfrd_map, spaxel_area_kpc = sfr_map2(halphadc_map=ha_mapdc/1e17, z=z)

    # Radius Map
    r_map, radius_ratio, r_eff = radius_ratio_map(plateifu=plateifu)

    # Log(O/H)+12 - N2O2
    n2o2_ratiomap, n2o2_metalmap = n2o2_Z_map(nii6585_mapdc, oii3727_mapdc, oii3729_mapdc)

    # Log(O/H)+12 - O3N2
    o3n2_ratiomap, o3n2_metalmap = o3n2_metal_map(o3_5008_fluxmap=o3_5008_mapdc, nii6585_fluxmap=nii6585_mapdc, ha_fluxmap=ha_mapdc,hb_fluxmap=hb_mapdc)

    # Log(O/H)+12 - NII/Ha
    n2_ratiomap, n2_metalmap = n2_metal_map(nii6585_fluxmap=nii6585_mapdc, ha_fluxmap=ha_mapdc)

    # Pipe3D Maps
    age_l, age_m, age_err, metal_l, metal_m, metal_err, stel_vel, vel_err, v_disp, v_err, ml_ratio, mass_rho, d4000_index, d4000_err, hdelta_index, hdelta_err = pipe3d_maps(plateifu, sample=sample)

    # Bluck et al. 2020 delta SFR = SFRD - Bluck least square minimization 
    sfms_fit_map = 0.90 * mass_rho - 9.57 # Bluck least square minimization fit
    delta_sfr = sfrd_map - sfms_fit_map 


    # Write Maps to FITS file
    #Writes a new fits for the data
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(ha_map_clean, ver=1, name='Halpha')) # Halpha [1e-17 erg / (cm2 s spaxel)]
    new_hdul.append(fits.ImageHDU(ha_mapdc, name='Halpha Kcorr',ver=2)) # Halpha k-corrected [1e-17 erg / (cm2 s spaxel)]
    new_hdul.append(fits.ImageHDU(snr_map.value, ver=3, name='SNR')) # SNR
    new_hdul.append(fits.ImageHDU(bdec_map, ver=4, name='Ha/Hb')) # Balmer decerment
    new_hdul.append(fits.ImageHDU(l_ha, ver=5, name='Ha Lum')) # erg / (s spaxel)
    new_hdul.append(fits.ImageHDU(sfr_map, ver=6, name='SFR')) # Msolar/yr
    new_hdul.append(fits.ImageHDU(sfrd_map, ver=7, name='SFR Density')) #Msolar/yr/kpc^2
    new_hdul.append(fits.ImageHDU(r_map, ver=8, name='Ellip R')) # arcsec
    new_hdul.append(fits.ImageHDU(radius_ratio, ver=9, name='R/Reff'))
    new_hdul.append(fits.ImageHDU(n2o2_metalmap, ver=10, name='Log(O/H)+12_[N2O2]'))
    new_hdul.append(fits.ImageHDU(o3n2_metalmap, ver=11, name='Log(O/H)+12_[O3N2]'))
    new_hdul.append(fits.ImageHDU(n2_metalmap, ver=12, name='Log(O/H)+12_[N2]'))
    new_hdul.append(fits.ImageHDU(age_l, ver=13, name='Gyr_lw'))
    new_hdul.append(fits.ImageHDU(age_m, ver=14, name='Gyr_mw'))
    new_hdul.append(fits.ImageHDU(age_err, ver=15, name='Gyr_err'))
    new_hdul.append(fits.ImageHDU(metal_l, ver=16, name='SP_ZsubL'))
    new_hdul.append(fits.ImageHDU(metal_m, ver=17, name='SP_ZsubM'))
    new_hdul.append(fits.ImageHDU(metal_err, ver=18, name='SP_Zerr'))
    new_hdul.append(fits.ImageHDU(stel_vel, ver=19, name='Vel_km/s'))
    new_hdul.append(fits.ImageHDU(vel_err, ver=20, name='Vel_err]'))
    new_hdul.append(fits.ImageHDU(v_disp, ver=21, name='Vdisp_km/s'))
    new_hdul.append(fits.ImageHDU(v_err, ver=22, name='Vdisp_err'))
    new_hdul.append(fits.ImageHDU(ml_ratio, ver=23, name='M/L'))
    new_hdul.append(fits.ImageHDU(mass_rho, ver=24, name='Msun/spx2'))
    new_hdul.append(fits.ImageHDU(mass_rho/0.25, ver=25, name='Msun/arcs2'))
    new_hdul.append(fits.ImageHDU(mass_rho/spaxel_area_kpc.value, ver=26, name='Msun/kpc2'))
    new_hdul.append(fits.ImageHDU(d4000_index, ver=27, name='D4000'))
    new_hdul.append(fits.ImageHDU(d4000_err, ver=28, name='D4000_err'))
    new_hdul.append(fits.ImageHDU(hdelta_index, ver=29, name='Hdelta'))
    new_hdul.append(fits.ImageHDU(hdelta_err, ver=30, name='Hdelta_err'))


    # Update header
    prihdr = new_hdul[0].header
    prihdr['SPX_AREA']='{} Kpc^2'.format(str(spaxel_area_kpc.value)[:5])
    prihdr['REFF']='{} ARCSECS'.format(str(r_eff)[:5])
    prihdr['z']='{} redshift'.format(str(z)[:5])
    prihdr['plateifu']=plateifu

    # Write data to FITS file
    new_hdul.writeto('/Users/mmckay/Desktop/research/FMR_MZR/{}_result_fits/{}_results.fits'.format(sample, plateifu), overwrite=True)

# Generate FITS files with IFU maps

In [ ]:
# bbrd sample
for plateifu in ['8550-12703', '8312-12704', '10001-3702', '8254-1902', '8595-3703']:
    #print(plateifu)
    fits_maps = write_maps2fits(plateifu, mode='local', sample='bbrd')
    # Check '8465-9102' - not working - is it downloaded locally?

In [ ]:
#Lg12 sample
lg12_plateifu_df = pd.read_csv('/Users/mmckay/Desktop/research/FMR_MZR/lg12_manga_table.csv')
for plateifu in lg12_plateifu_df['plateifu']:
    #print(plateifu)
    try: # Some files were not found due to missing pipe3d fits files 
        fits_maps = write_maps2fits(plateifu, mode='local', sample='lg12')
    except:
        print(plateifu)

# Make IFU maps in 1D columns and stores in pandas dataframe for each galaxy

In [ ]:
# Input 2d data as a 1d Series in a dataframe
# 1. Interable Function to read in FITS file extensions - done
# 2. Flatten 2D maps array to 1D column array
# 3. Add Columns to pandas DataFrame
# 4. Append each galaxy dataframe to combo dataframe

In [ ]:
# Read FITS file extension 
def read_fits_ext(fits_file, ext=1):
    hdu = fits.open(fits_file)
    sci_maps = hdu[ext].data
    hdu.close()
    return sci_maps

In [ ]:
# Store 2D Maps as 1D columns in a pandas dataframe
# List of BBRD FITS files
fits_filelist = glob.glob('/Users/mmckay/Desktop/research/FMR_MZR/bbrd_result_fits/*.fits')
fits_filelist

# 
extnum_list = np.arange(1,31,1) # The range of FITS extension
for fit in fits_filelist:
    hdu = fits.open(fit)
    ifu_df = pd.DataFrame()
    for extnum in extnum_list:
        # Read in 2d data from fits file extension 
        map2d_data = read_fits_ext(fit, ext=hdu[extnum-1].name)
        #Flatten 2d map
        map1d_data = map2d_data.flatten(order='C')
        #print(extnum, hdu[extnum-1].ver, hdu[extnum-1].name, map1d_data.shape)
        # Pair extname column with the 1D Map data
        ifu_df[hdu[extnum-1].name] = map1d_data
        
    print(ifu_df.shape)
    ifu_df.to_csv(path_or_buf='/Users/mmckay/Desktop/research/FMR_MZR/bbrd_result_fits/{}_map.csv'.format(hdu[0].header['plateifu']), sep=',')
    hdu.close()

In [ ]:
# LG12
# Store 2D Maps as 1D columns in a pandas dataframe
# List of BBRD FITS files
fits_filelist = glob.glob('/Users/mmckay/Desktop/research/FMR_MZR/lg12_result_fits/*.fits')
fits_filelist

# 
extnum_list = np.arange(1,31,1) # The range of FITS extension
for fit in fits_filelist:
    hdu = fits.open(fit)
    ifu_df = pd.DataFrame()
    for extnum in extnum_list:
        # Read in 2d data from fits file extension 
        map2d_data = read_fits_ext(fit, ext=hdu[extnum-1].name)
        #Flatten 2d map
        map1d_data = map2d_data.flatten(order='C')
        #print(extnum, hdu[extnum-1].ver, hdu[extnum-1].name, map1d_data.shape)
        # Pair extname column with the 1D Map data
        ifu_df[hdu[extnum-1].name] = map1d_data
        
    #print(ifu_df.shape)
    ifu_df.to_csv(path_or_buf='/Users/mmckay/Desktop/research/FMR_MZR/lg12_result_fits/{}_map.csv'.format(hdu[0].header['plateifu']), sep=',')
    hdu.close()

# Combine all dataframes in the sample into a super dataframe

In [ ]:
# Combine all dataframes into a super dataframe
bbrd_csv_list = glob.glob('/Users/mmckay/Desktop/research/FMR_MZR/bbrd_result_fits/*_map.csv')
csv_list = []
for bbrd in bbrd_csv_list:
    bbrd_df = pd.read_csv(bbrd)
    print(bbrd, bbrd_df.shape)
    csv_list.append(bbrd_df)

#len(csv_list)
bbrd_supercsv = pd.concat(csv_list)
bbrd_supercsv.to_csv(path_or_buf='/Users/mmckay/Desktop/research/FMR_MZR/bbrd_result_fits/bbrd_mastertable.csv', sep=',')

print(bbrd_supercsv.shape)
bbrd_supercsv.head(10)
# Replace inf value with nans
#bbrd_supercsv_clean = bbrd_supercsv.replace([np.inf, -np.inf], np.nan)

#pd.set_option('use_inf_as_na', True) # compute inf values as nans

# LG12 sample
# Combine all dataframes into a super dataframe
lg12_csv_list = glob.glob('/Users/mmckay/Desktop/research/FMR_MZR/lg12_result_fits/*_map.csv')
csv_list = []
for csv in lg12_csv_list:
    csv_df = pd.read_csv(csv)
    print(csv, bbrd_df.shape)
    csv_list.append(csv_df)

#len(csv_list)
lg12_supercsv = pd.concat(csv_list)
lg12_supercsv.to_csv(path_or_buf='/Users/mmckay/Desktop/research/FMR_MZR/lg12_result_fits/lg12_mastertable.csv', sep=',')

print(lg12_supercsv.shape)
lg12_supercsv.head(10)

In [ ]:
pd.read_csv('/Users/mmckay/Desktop/research/FMR_MZR/lg12_result_fits/lg12_mastertable.csv', sep=',')

# Testing

In [ ]:
# Retrieve the data from FITS file
plateifu = '8550-12703'
sample = 'bbrd'
hdu_8550_12703 = fits.open('/Users/mmckay/Desktop/research/FMR_MZR/{}_result_fits/{}_results.fits'.format(sample, plateifu))
hdu_8550_12703.info()

In [ ]:
hdu_8550_12703[0].header['*']

In [ ]:
hdu_8550_12703[3].ver

In [ ]:
#maps = Maps(plateifu='8465-9102', mode='local')

In [ ]:
#write_maps2fits('8465-9102', mode='remote')